### Intall libraries

In [1]:
from IPython.display import clear_output

!pip install datasets
!pip install scikit-learn
!pip install RapidFuzz
clear_output(wait=False)

### Evaluate

Dataset chosen for evaluation: [vishnun/SpellGram](https://huggingface.co/datasets/vishnun/SpellGram)

Load eval dataset

In [2]:
from datasets import load_dataset

ds = load_dataset("vishnun/SpellGram", trust_remote_code=True)
ds = ds["train"].shuffle(seed=42).select(range(2000))

m:\python_projects\context_sensitive_spelling_correction\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ds

Dataset({
    features: ['source', 'target'],
    num_rows: 2000
})

In [4]:
ds[0]

{'source': 'i havv arranged a conference call to oiscuss these matters further',
 'target': 'i have arranged a conference call to discuss these matters further'}

Load spelling corrector

In [5]:
from spelling_corrector import SpellCorrector


ngram_model_path = r"..\models\trigram_model_shakespeare.pkl"
spell_corrector = SpellCorrector(ngram_model_path, n=3)
print(spell_corrector.get_vocab_size())

sentence = "i sleep at nite"
corrected_sentence = spell_corrector.correct_sentence(sentence)
print("  initial: ", sentence)
print("corrected: ", corrected_sentence)

12072
  initial:  i sleep at nite
corrected:  i sleep at unite


In [6]:
from spelling_corrector import SpellCorrectorImproved

ngram_model_impr_path = r"..\models\trigram_model_bookcorpus_.pkl"
spell_corrector_impr = SpellCorrectorImproved(ngram_model_impr_path, n=3)

corrected_sentence = spell_corrector_impr.correct_sentence(sentence)
print("  initial: ", sentence)
print("corrected: ", corrected_sentence)

  initial:  i sleep at nite
corrected:  i sleep at nite


Evaluation of baseline model

In [7]:
from rapidfuzz.distance import Levenshtein
from tqdm import tqdm


def evaluate_spelling_corrector(corrector, test_data):
    total_words = 0
    correct_words = 0
    total_chars = 0
    total_edit_distance = 0
    
    for example in tqdm(test_data):
        source = example["source"]
        target = example["target"]
        corrected = corrector.correct_sentence(source)
        
        # Word Accuracy
        target_words = target.split()
        corrected_words = corrected.split()
        
        total_words += len(target_words)
        correct_words += sum(1 for tw, cw in zip(target_words, corrected_words) if tw == cw)
        
        # Character Error Rate (CER)
        total_chars += len(target)
        total_edit_distance += Levenshtein.distance(target, corrected)

    word_accuracy = correct_words / total_words if total_words > 0 else 0
    cer = total_edit_distance / total_chars if total_chars > 0 else 0

    return {"word_accuracy": word_accuracy, "character_error_rate": cer}

In [8]:
result = evaluate_spelling_corrector(spell_corrector, ds)

print(f"Word accuracy: {result['word_accuracy']:.4f}")
print(f"Character error rate: {result['character_error_rate']:.4f}")

100%|██████████| 2000/2000 [05:51<00:00,  5.69it/s]

Word accuracy: 0.7078
Character error rate: 0.0782


Evaluation of improved model

In [8]:
result = evaluate_spelling_corrector(spell_corrector_impr, ds)

print(f"Word accuracy: {result['word_accuracy']:.4f}")
print(f"Character error rate: {result['character_error_rate']:.4f}")

100%|██████████| 2000/2000 [09:41<00:00,  3.44it/s]

Word accuracy: 0.8143
Character error rate: 0.0535
